In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd '/content/gdrive/MyDrive/IOT Intrusion Detection'

/content/gdrive/MyDrive/IOT Intrusion Detection


In [1]:
import pandas as pd
import numpy as np
import pickle

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [5]:
import tensorflow as tf

In [6]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.layers import LSTM, GRU, SimpleRNN

In [7]:
 pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 3.6 MB/s eta 0:00:00


In [8]:
import category_encoders as ce

In [9]:
df = pd.read_csv('/content/gdrive/MyDrive/IOT Intrusion Detection/IoT Network Intrusion Dataset.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625783 entries, 0 to 625782
Data columns (total 86 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow_ID            625783 non-null  object 
 1   Src_IP             625783 non-null  object 
 2   Src_Port           625783 non-null  int64  
 3   Dst_IP             625783 non-null  object 
 4   Dst_Port           625783 non-null  int64  
 5   Protocol           625783 non-null  int64  
 6   Timestamp          625783 non-null  object 
 7   Flow_Duration      625783 non-null  int64  
 8   Tot_Fwd_Pkts       625783 non-null  int64  
 9   Tot_Bwd_Pkts       625783 non-null  int64  
 10  TotLen_Fwd_Pkts    625783 non-null  float64
 11  TotLen_Bwd_Pkts    625783 non-null  float64
 12  Fwd_Pkt_Len_Max    625783 non-null  float64
 13  Fwd_Pkt_Len_Min    625783 non-null  float64
 14  Fwd_Pkt_Len_Mean   625783 non-null  float64
 15  Fwd_Pkt_Len_Std    625783 non-null  float64
 16  Bw

In [11]:
df.head()

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25/07/2019 03:25:53 AM,75,1,1,...,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,Anomaly,Mirai,Mirai-Ackflooding
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,26/05/2019 10:11:06 PM,5310,1,2,...,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,Anomaly,DoS,DoS-Synflooding
2,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11/07/2019 01:24:48 AM,141,0,3,...,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,Anomaly,Scan,Scan Port OS
3,192.168.0.13-192.168.0.16-9020-52964-6,192.168.0.16,52964,192.168.0.13,9020,6,04/09/2019 03:58:17 AM,151,0,2,...,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,Anomaly,Mirai,Mirai-Hostbruteforceg
4,192.168.0.1-239.255.255.250-36763-1900-17,192.168.0.1,36763,239.255.255.250,1900,17,10/09/2019 01:41:18 AM,153,2,1,...,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,Anomaly,Mirai,Mirai-Hostbruteforceg


In [12]:
df = df.drop(['Flow_ID','Src_IP','Dst_IP', 'Timestamp'], axis = 1)

In [13]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(axis=0)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625415 entries, 0 to 625782
Data columns (total 82 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Src_Port           625415 non-null  int64  
 1   Dst_Port           625415 non-null  int64  
 2   Protocol           625415 non-null  int64  
 3   Flow_Duration      625415 non-null  int64  
 4   Tot_Fwd_Pkts       625415 non-null  int64  
 5   Tot_Bwd_Pkts       625415 non-null  int64  
 6   TotLen_Fwd_Pkts    625415 non-null  float64
 7   TotLen_Bwd_Pkts    625415 non-null  float64
 8   Fwd_Pkt_Len_Max    625415 non-null  float64
 9   Fwd_Pkt_Len_Min    625415 non-null  float64
 10  Fwd_Pkt_Len_Mean   625415 non-null  float64
 11  Fwd_Pkt_Len_Std    625415 non-null  float64
 12  Bwd_Pkt_Len_Max    625415 non-null  float64
 13  Bwd_Pkt_Len_Min    625415 non-null  float64
 14  Bwd_Pkt_Len_Mean   625415 non-null  float64
 15  Bwd_Pkt_Len_Std    625415 non-null  float64
 16  Fl

In [15]:
df[['Label','Cat','Sub_Cat']].nunique()

Label      2
Cat        5
Sub_Cat    9
dtype: int64

In [16]:
redundant_feat = list(df.loc[:, df.nunique() == 1].columns)

In [17]:
df = df.drop(redundant_feat, axis = 1)

In [18]:
df['Cat'].value_counts()

Mirai                415309
Scan                  75265
DoS                   59391
Normal                40073
MITM ARP Spoofing     35377
Name: Cat, dtype: int64

In [19]:
df['Label'].value_counts()

Anomaly    585342
Normal      40073
Name: Label, dtype: int64

In [44]:
labelencoder = LabelEncoder()
df['Label']= labelencoder.fit_transform(df['Label'])

In [45]:
X = df.iloc[:,0:69]
y = df['Label']

In [46]:
y = tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32'
)

In [47]:
d1 = pd.DataFrame(y)

In [48]:
d1.head()

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [49]:
X[['Src_Port','Dst_Port','Protocol']].nunique()

Src_Port    11205
Dst_Port     1034
Protocol        3
dtype: int64

In [50]:
X['Src_Port'].unique()

array([10000,  2179, 52727, ..., 63997, 51806,  4656])

In [51]:
encoder = ce.BinaryEncoder(cols = ['Src_Port','Dst_Port','Protocol'])

In [52]:
X = encoder.fit_transform(X)

In [53]:
scaler = MinMaxScaler()

In [54]:
X= scaler.fit_transform(X)
# X_test = scaler.fit_transform(X_test)

Split the data

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 100)

Covariance Matrix

In [56]:
import numpy as np

In [57]:
cm = np.cov(X.T)

In [58]:
cm

array([[ 2.19696008e-02, -1.10955003e-03,  2.88331697e-03, ...,
         4.02469080e-05,  6.70322497e-04,  6.31599140e-04],
       [-1.10955003e-03,  4.69316138e-02,  2.01761130e-02, ...,
         2.92015119e-05,  1.36574073e-03,  1.34379876e-03],
       [ 2.88331697e-03,  2.01761130e-02,  5.64860525e-02, ...,
         4.44608293e-05,  1.47983328e-03,  1.44112900e-03],
       ...,
       [ 4.02469080e-05,  2.92015119e-05,  4.44608293e-05, ...,
         2.95781855e-04,  3.55712316e-04,  1.39498962e-05],
       [ 6.70322497e-04,  1.36574073e-03,  1.47983328e-03, ...,
         3.55712316e-04,  8.22589911e-04,  3.89190615e-04],
       [ 6.31599140e-04,  1.34379876e-03,  1.44112900e-03, ...,
         1.39498962e-05,  3.89190615e-04,  3.73636106e-04]])

In [59]:
eig_val, eig_vac = np.linalg.eig(cm)

In [39]:
eig_val.shape

(93,)

In [40]:
import plotly.express as px

In [41]:
from sklearn.decomposition import PCA

**LDA**

In [ ]:
pca = PCA()
pca.fit(X)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)

100% cut-off threshold so we considered 32 components

### Create PCA model with 32 components

In [ ]:
pcamodel = PCA(n_components=10)

In [ ]:
x_train_new = pcamodel.fit_transform(X_train)
x_test_new = pcamodel.fit_transform(X_test)

In [ ]:
x_train_new.shape

(500332, 10)

In [ ]:
# ANN MODEL
ann = Sequential()
ann.add(Dense(units=128, input_dim=10, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=256, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=512, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=256, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=128, activation='relu'))
ann.add(Dropout(0.5))
# ann.add(Dense(units=4096,activation="relu"))
# ann.add(Dense(units=4096,activation="relu"))
ann.add(Dense(units=y_train.shape[1], activation='softmax'))

In [ ]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.001)

In [ ]:
ann.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [ ]:
ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1408      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 256)               1

In [ ]:
# ann.fit(x_train_new, y_train)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint("Vgg16_1.h5", monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early_stop = EarlyStopping(monitor='accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
callbacks_list = [early_stop, checkpoint]

In [ ]:
ann.fit(x_train_new, y_train,verbose=2,epochs=50,batch_size=64,callbacks = callbacks_list)

Epoch 1/50

Epoch 1: accuracy improved from -inf to 0.99079, saving model to Vgg16_1.h5
7818/7818 - 26s - loss: 0.0360 - accuracy: 0.9908 - 26s/epoch - 3ms/step
Epoch 2/50

Epoch 2: accuracy improved from 0.99079 to 0.99459, saving model to Vgg16_1.h5
7818/7818 - 23s - loss: 0.0238 - accuracy: 0.9946 - 23s/epoch - 3ms/step
Epoch 3/50

Epoch 3: accuracy improved from 0.99459 to 0.99479, saving model to Vgg16_1.h5
7818/7818 - 23s - loss: 0.0219 - accuracy: 0.9948 - 23s/epoch - 3ms/step
Epoch 4/50

Epoch 4: accuracy improved from 0.99479 to 0.99485, saving model to Vgg16_1.h5
7818/7818 - 23s - loss: 0.0227 - accuracy: 0.9948 - 23s/epoch - 3ms/step
Epoch 5/50

Epoch 5: accuracy did not improve from 0.99485
7818/7818 - 22s - loss: 0.0241 - accuracy: 0.9947 - 22s/epoch - 3ms/step
Epoch 6/50

Epoch 6: accuracy did not improve from 0.99485
7818/7818 - 22s - loss: 0.0234 - accuracy: 0.9946 - 22s/epoch - 3ms/step
Epoch 7/50

Epoch 7: accuracy did not improve from 0.99485
7818/7818 - 24s - loss: 

In [ ]:
ann.evaluate(x_test_new, y_test)

3909/3909 [==============================] - 10s 2ms/step - loss: 0.0159 - accuracy: 0.9965


[0.015883926302194595, 0.9964823126792908]

In [ ]:
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,confusion_matrix)

In [ ]:
y_test_arg=np.argmax(y_test,axis=1)

In [ ]:
Y_pred = np.argmax(ann.predict(x_test_new),axis=1)

3909/3909 [==============================] - 6s 2ms/step


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test_arg, Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    116989
           1       0.98      0.96      0.97      8094

    accuracy                           1.00    125083
   macro avg       0.99      0.98      0.99    125083
weighted avg       1.00      1.00      1.00    125083



In [ ]:
print('Confusion Matrix')
print(confusion_matrix(y_test_arg, Y_pred))

Confusion Matrix
[[116860    129]
 [   311   7783]]
